<a href="https://colab.research.google.com/github/NaviDSX/2-S1/blob/main/DETR/BFinal_Fine_tuning_DetrForSegmentation_on_custom_dataset_end_to_end_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git timm

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 376 kB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 39.3 MB/s 
     |████████████████████████████████| 636 kB 52.4 MB/s 
     |████████████████████████████████| 895 kB 34.9 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 


In [2]:
!pip install -q pytorch-lightning

     |████████████████████████████████| 925 kB 5.5 MB/s 
     |████████████████████████████████| 829 kB 36.1 MB/s 
     |████████████████████████████████| 125 kB 43.7 MB/s 
     |████████████████████████████████| 282 kB 45.8 MB/s 
     |████████████████████████████████| 1.3 MB 34.9 MB/s 
     |████████████████████████████████| 160 kB 45.8 MB/s 
     |████████████████████████████████| 271 kB 43.0 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
torch.set_grad_enabled(False);
import json
from pathlib import Path
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

class CocoPanoptic(torch.utils.data.Dataset):
    def __init__(self, img_folder,feature_extractor):
        self.img_folder = img_folder
        self.feature_extractor = feature_extractor
        self.idfn = [x for x in os.listdir(img_folder)]

    def __getitem__(self, idx):
        img_path = Path(self.img_folder)/self.idfn[idx].replace('.png', '.jpg')
        img = Image.open(img_path).convert('RGB')
        
        # preprocess image and target (converting target to DETR format, resizing + normalization of both image and target)
        encoding = self.feature_extractor(images=img, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze() # remove batch dimension
        #target = encoding["labels"][0] 
        return pixel_values

    def __len__(self):
        return len(self.idfn)

In [5]:
import os
from transformers import DetrFeatureExtractor
import numpy as np
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50-panoptic", size=500, max_size=600)
dataset2 = CocoPanoptic(img_folder='/content/drive/MyDrive/coco/val2017',feature_extractor=feature_extractor)

Downloading:   0%|          | 0.00/273 [00:00<?, ?B/s]

In [6]:
from torch.utils.data import DataLoader

def collate_fn2(batch):
  pixel_values = [item for item in batch]
  encoded_input = feature_extractor.pad_and_create_pixel_mask(pixel_values, return_tensors="pt")
  #labels = [item[1] for item in batch]
  batch = {}
  batch['pixel_values'] = encoded_input['pixel_values']
  batch['pixel_mask'] = encoded_input['pixel_mask']
  #batch['labels'] = labels
  return batch

In [7]:
from transformers import DetrConfig, DetrForSegmentation
model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")

Downloading:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/164M [00:00<?, ?B/s]

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50_ram-a26f946b.pth" to /root/.cache/torch/hub/checkpoints/resnet50_ram-a26f946b.pth


In [8]:
def osize(A):
    ridx,cidx=A.shape
    for x in reversed(range(cidx-1)):
        if torch.all(A[:,cidx-1] == 0):
            cidx=cidx-1
        else:
            break
    for x in reversed(range(ridx-1)):
        if torch.all(A[ridx-1,:] == 0):
            ridx=ridx-1
        else:
            break
    return (ridx,cidx)

In [9]:
import time
time.ctime()

'Fri Oct  8 07:03:35 2021'

In [10]:
import gc
gc.collect()

1295

In [11]:
dataloader2 = DataLoader(dataset2, collate_fn=collate_fn2, batch_size=5, shuffle=True)

In [12]:
from tqdm.notebook import tqdm
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_grad_enabled(False);

model.to(device)
model.eval()
print(time.ctime())
print("Running evaluation...")
li=[]
for idx, batch in enumerate(tqdm(dataloader2)):
    # get the inputs
    print(f'idx is {idx}')
    pixel_values = batch["pixel_values"].to(device)
    pixel_mask = batch["pixel_mask"].to(device)
    print(pixel_values.shape,pixel_mask.shape)
    #print(batch["pixel_mask"].shape)
    ots = torch.stack([torch.as_tensor(osize(pixel_mask[i,:,:])) for i in range(len(pixel_mask))], dim=0)
    #labels = [{k: v.to(device) for k, v in t.items()} for t in batch["labels"]] # these are in DETR format, resized + normalized

    # forward pass
    with torch.no_grad():
      outputs = model(pixel_values=pixel_values, pixel_mask=pixel_mask)
      print(outputs['logits'].shape, outputs['pred_boxes'].shape, outputs['pred_masks'].shape)

    # # object detection evaluation
    # orig_target_sizes = torch.stack([target["orig_size"] for target in labels], dim=0)
    # results = feature_extractor.post_process(outputs, orig_target_sizes) # convert outputs of model to COCO api
    # target_sizes = torch.stack([t["size"] for t in labels], dim=0)
    # results = feature_extractor.post_process_segmentation(results, outputs, orig_target_sizes, target_sizes)
    # res = {target['image_id'].item(): output for target, output in zip(labels, results)}
    # coco_evaluator.update(res)

    # panoptic segmentation evaluation
    #target_sizes = torch.stack([t["size"] for t in labels], dim=0)
    results_panoptic = feature_extractor.post_process_panoptic(outputs,ots) # convert outputs of model to COCO api
    navi=(pixel_values,pixel_mask,ots,outputs,results_panoptic)
    li.append(navi)
    print(time.ctime())

    #import torch

    # setting device on GPU if available, else CPU
    #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)
    print()

    #Additional Info when using cuda
    if device.type == 'cuda':
        print(torch.cuda.get_device_name(0))
        print('Memory Usage:')
        print('Allocated:      ', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Reserved/Cached:', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
#     for i, target in enumerate(labels):
#         image_id = target["image_id"].item()
#         file_name = f"{image_id:012d}.png"
#         results_panoptic[i]["image_id"] = image_id
#         results_panoptic[i]["file_name"] = file_name
#     panoptic_evaluator.update(results_panoptic)

# coco_evaluator.synchronize_between_processes()
# coco_evaluator.accumulate()
# coco_evaluator.summarize()

Running evaluation...


  0%|          | 0/1000 [00:00<?, ?it/s]

idx is 0
torch.Size([5, 3, 600, 600]) torch.Size([5, 600, 600])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


torch.Size([5, 100, 251]) torch.Size([5, 100, 4]) torch.Size([5, 100, 150, 150])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Using device: cuda

Tesla K80
Memory Usage:
Allocated: 0.2 GB
Cached:    9.8 GB


/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:375: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


idx is 1
torch.Size([5, 3, 599, 600]) torch.Size([5, 599, 600])
torch.Size([5, 100, 251]) torch.Size([5, 100, 4]) torch.Size([5, 100, 150, 150])
Using device: cuda

Tesla K80
Memory Usage:
Allocated: 0.3 GB
Cached:    8.5 GB
idx is 2
torch.Size([5, 3, 600, 599]) torch.Size([5, 600, 599])
torch.Size([5, 100, 251]) torch.Size([5, 100, 4]) torch.Size([5, 100, 150, 150])
Using device: cuda

Tesla K80
Memory Usage:
Allocated: 0.4 GB
Cached:    8.5 GB
idx is 3
torch.Size([5, 3, 600, 600]) torch.Size([5, 600, 600])
torch.Size([5, 100, 251]) torch.Size([5, 100, 4]) torch.Size([5, 100, 150, 150])
Using device: cuda

Tesla K80
Memory Usage:
Allocated: 0.5 GB
Cached:    8.5 GB
idx is 4
torch.Size([5, 3, 450, 600]) torch.Size([5, 450, 600])
torch.Size([5, 100, 251]) torch.Size([5, 100, 4]) torch.Size([5, 100, 113, 150])
Using device: cuda

Tesla K80
Memory Usage:
Allocated: 0.5 GB
Cached:    8.5 GB
idx is 5
torch.Size([5, 3, 600, 600]) torch.Size([5, 600, 600])
torch.Size([5, 100, 251]) torch.Size

RuntimeError: ignored

In [ ]:
hello+5

As we can see, to initialize the dataset, 3 things need to be provided besides the feature extractor:
* the path to a directory containing the images.
* the path to a directory containing the segmentation masks (PNG files).
* the path to the annotation file (JSON), containing the annotations in COCO format.

You can download the data from the [official website](https://cocodataset.org/#download) (under "Dataset" -> "Download" -> "2017 Val images [5K/1GB]" and "2017 Panoptic Train/Val annotations [821MB]"). I've downloaded the data, and stored it in my personal Google Drive.

As the training dataset is very big, we use Numpy to randomly pick a small portion of it, for demonstration purposes.

In [ ]:
from transformers import DetrFeatureExtractor
import numpy as np

# we reduce the size and max_size to be able to fit the batches in GPU memory
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50-panoptic", size=500, max_size=600)

dataset = CocoPanoptic(img_folder='/content/drive/MyDrive/coco/val2017', 
                             ann_folder='/content/drive/MyDrive/coco/panoptic_val2017',
                             ann_file='/content/drive/MyDrive/coco/panoptic_val2017.json',
                             feature_extractor=feature_extractor)

# let's split it up into very tiny training and validation sets using random indices
np.random.seed(42)
indices = np.random.randint(low=0, high=len(dataset), size=50)
train_dataset = torch.utils.data.Subset(dataset, indices[:40])
val_dataset = torch.utils.data.Subset(dataset, indices[40:])

In [ ]:
feature_extractor

In [ ]:
vars(feature_extractor)

In [ ]:
dataset[0]

In [ ]:
type(encoding)

In [ ]:
pixel_values, target = train_dataset[2]
print(pixel_values.shape)
print(target.keys())

We pick 40 examples for training, and 10 for validation.

In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(val_dataset))

Next, let's create corresponding dataloaders. We define a custom `collate_fn`, which will batch the images (and labels) together. As images can have different sizes, images in a batch are padded up to the largest one, and a `pixel_mask` is created, indicating which pixels are real/which are padding. We can achieve this using the `pad_and_create_pixel_mask` method of `DetrFeatureExtractor`.

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
  pixel_values = [item[0] for item in batch]
  encoded_input = feature_extractor.pad_and_create_pixel_mask(pixel_values, return_tensors="pt")
  labels = [item[1] for item in batch]
  batch = {}
  batch['pixel_values'] = encoded_input['pixel_values']
  batch['pixel_mask'] = encoded_input['pixel_mask']
  batch['labels'] = labels
  return batch

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=3, shuffle=True)
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=10)

In [ ]:
type(train_dataset[0])

In [ ]:
c=next(iter(train_dataloader))

In [ ]:
type(c)

In [ ]:
len(train_dataset[0])

In [ ]:
train_dataset[0][0], type(train_dataset[0][0])

In [ ]:
train_dataset[0][1], type(train_dataset[0][1])

In [ ]:
train_dataset[0][1].keys()

In [ ]:
dataloader2 = DataLoader(dataset2, collate_fn=collate_fn2, batch_size=20, shuffle=True)

In [ ]:
batch.keys()

In [ ]:
dataset2[0]

In [ ]:
next(iter(dataloader2))

In [ ]:
type(dataset2[0])

In [ ]:
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=10)

In [ ]:
import gc
gc.collect()

In [ ]:
len(outputs)

In [ ]:
outputs[0]

In [ ]:
len(results_panoptic)

In [ ]:
results_panoptic[0]

In [ ]:
type

In [ ]:
j=pixel_mask.to('cpu').numpy()

In [ ]:
j=pixel_mask

In [ ]:
j.shape

In [ ]:
k=j[0,:,:]

In [ ]:
k.shape

In [ ]:
l=j[1,:,:]

In [ ]:
m=torch.dstack((k,l))

In [ ]:
osize(l)

In [ ]:
m.shape

In [ ]:
orig_target_sizes = torch.stack([j[i,:,:] for i in range(2)], dim=0)

In [ ]:
orig_target_sizes.shape

In [ ]:
orig_target_sizes = torch.stack([j[i,:,:] for i in range(2)], dim=0)

In [ ]:
ots = torch.stack([torch.as_tensor(osize(j[i,:,:])) for i in range(2)], dim=0)

In [ ]:
ots.shape

In [ ]:


for x in pixel_mask:
  print(x)

In [ ]:
torch.as_tensor(f)

In [ ]:
f=(1,2)

In [ ]:
g=torch.as_tensor(f)

In [ ]:
g.shape

In [ ]:
d=[[1],[2]]

In [ ]:
g=torch.as_tensor(d)

In [ ]:
g.shape

In [ ]:
len(pixel_mask)

In [ ]:
ots = torch.stack([torch.as_tensor(osize(j[i,:,:])) for i in range(len(pixel_mask))], dim=0)

In [ ]:
ots.shape

In [ ]:
pixel_values.shape, pixel_mask.shape

In [ ]:
batch["pixel_values"].shape

In [ ]:
type(outputs), len(outputs)

In [ ]:
outputs.keys()

In [ ]:
outputs['logits'].shape, outputs['pred_boxes'].shape, outputs['pred_masks'].shape

Shape of pixel_values: torch.Size([3, 599, 402])
Shape of logits: torch.Size([3, 100, 251])
Shape of predicted bounding boxes: torch.Size([3, 100, 4])
Shape of predicted masks: torch.Size([3, 100, 125, 150])
This looks ok: the logits are of shape (batch_size, num_queries, num_classes + 1) - + 1 because there's also a "no object" class. The predicted bounding boxes are of shape (batch_size, num_queries, 4), since each bounding box consists of 4 coordinates (DETR outputs them in center_x, center_y, width, height format). The predicted masks logits are of shape (batch_size, num_queries, height/4, width/4).

## Train the model using PyTorch Lightning

Here we define a LightningModule, which is an `nn.Module` with some extra functionality. For a good introduction to PyTorch Lightning, I refer to its [documentation](https://pytorch-lightning.readthedocs.io/en/latest/?_ga=2.260781005.179149119.1623936252-1738348008.1615553774) as well as the official [tutorials](https://pytorch-lightning.readthedocs.io/en/latest/#tutorials).

Basically, PyTorch Lightning takes care of all the boilerplate such as defining the training loop, placing tensors on the right devices, fetching data from the dataloaders, and so on. We just need to define the forward pass, the training and validation step.

In [ ]:
import pytorch_lightning as pl
import torch

class DetrPanoptic(pl.LightningModule):

     def __init__(self, model, lr, lr_backbone, weight_decay):
         super().__init__()
      
         self.model = model

         # see https://github.com/PyTorchLightning/pytorch-lightning/pull/1896
         self.lr = lr
         self.lr_backbone = lr_backbone
         self.weight_decay = weight_decay

     def forward(self, pixel_values, pixel_mask):
       outputs = self.model(pixel_values=pixel_values, pixel_mask=pixel_mask)

       return outputs
     
     def common_step(self, batch, batch_idx):
       pixel_values = batch["pixel_values"]
       pixel_mask = batch["pixel_mask"]
       labels = [{k: v.to(self.device) for k, v in t.items()} for t in batch["labels"]]

       outputs = self.model(pixel_values=pixel_values, pixel_mask=pixel_mask, labels=labels)

       loss = outputs.loss
       loss_dict = outputs.loss_dict

       return loss, loss_dict

     def training_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)
        for k,v in loss_dict.items():
          self.log("train_" + k, v.item())

        return loss

     def validation_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss)
        for k,v in loss_dict.items():
          self.log("validation_" + k, v.item())

        return loss

     def configure_optimizers(self):
        param_dicts = [
              {"params": [p for n, p in self.named_parameters() if "backbone" not in n and p.requires_grad]},
              {
                  "params": [p for n, p in self.named_parameters() if "backbone" in n and p.requires_grad],
                  "lr": self.lr_backbone,
              },
        ]
        optimizer = torch.optim.AdamW(param_dicts, lr=self.lr,
                                  weight_decay=self.weight_decay)
        
        return optimizer

     def train_dataloader(self):
        return train_dataloader

     def val_dataloader(self):
        return val_dataloader

Let's start up Tensorboard, which will show us some nice plots over the course of training (PyTorch Lightning uses Tensorboard as default logger).

In [ ]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Here we load the model trained on COCO panoptic. We decide to only train the class labels classifier from scratch, and further fine-tune the bounding box regressor and mask head.

In [ ]:
from transformers import DetrConfig, DetrForSegmentation

model = DetrForSegmentation.from_pretrained("facebook/detr-resnet-50-panoptic")
state_dict = model.state_dict()
# Remove class weights
del state_dict["detr.class_labels_classifier.weight"]
del state_dict["detr.class_labels_classifier.bias"]
# define new model with custom class classifier
config = DetrConfig.from_pretrained("facebook/detr-resnet-50-panoptic", num_labels=250)
model.load_state_dict(state_dict, strict=False)

Next, we define the PyTorch LightningModule, and verify its outputs on a batch.

In [ ]:
model = DetrPanoptic(model=model, lr=1e-4, lr_backbone=1e-5, weight_decay=1e-4)

# pick the first training batch
batch = next(iter(train_dataloader))
# forward through the model
outputs = model(pixel_values=batch['pixel_values'], pixel_mask=batch['pixel_mask'])

In [ ]:
print("Shape of pixel_values:", pixel_values.shape)
print("Shape of logits:", outputs.logits.shape)
print("Shape of predicted bounding boxes:", outputs.pred_boxes.shape)
print("Shape of predicted masks:", outputs.pred_masks.shape)

This looks ok: the logits are of shape `(batch_size, num_queries, num_classes + 1)` - + 1 because there's also a "no object" class. The predicted bounding boxes are of shape `(batch_size, num_queries, 4)`, since each bounding box consists of 4 coordinates (DETR outputs them in center_x, center_y, width, height format). The predicted masks logits are of shape `(batch_size, num_queries, height/4, width/4)`.

Next, let's train! We train for a maximum of 25 epochs, and also use gradient clipping. You can refresh Tensorboard above to see how the losses evolve.

In [ ]:
from pytorch_lightning import Trainer

trainer = Trainer(gpus=1, max_epochs=25, gradient_clip_val=0.1)
trainer.fit(model)

Note that after training, PyTorch Lightning automatically saves the checkpoint in the logging directory. You can load it back in as follows:

In [ ]:
# model = DetrPanoptic.load_from_checkpoint(checkpoint_path="/content/lightning_logs/version_2/checkpoints/epoch=1-step=57.ckpt", 
#    

## Evaluate the model

Finally, we evaluate the model on the validation set. The original DETR repo has some nice evaluation tools that we will use (`CocoEvaluator` and `PanopticEvaluator`), which we are going to import.

In [ ]:
! rm -r detr
! git clone https://github.com/facebookresearch/detr.git
%cd detr

In [ ]:
! pip install -q git+https://github.com/cocodataset/panopticapi.git

We can use `CocoEvaluator` to evaluate the predicted bounding boxes + predicted instance segmentation masks. For this, we need load the ground truth COCO detection dataset. We only need to retrieve the original COCO annotations for the images included in our validation dataset.

In [ ]:
import torchvision
import os

class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(self, img_folder, ann_file, feature_extractor, train=True):
        super(CocoDetection, self).__init__(img_folder, ann_file)
        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        # read in PIL image and target in COCO format
        img, target = super(CocoDetection, self).__getitem__(idx)
        
        # preprocess image and target (converting target to DETR format, resizing + normalization of both image and target)
        image_id = self.ids[idx]
        target = {'image_id': image_id, 'annotations': target}
        encoding = self.feature_extractor(images=img, annotations=target, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze() # remove batch dimension
        target = encoding["labels"][0] # remove batch dimension

        return pixel_values, target

In [ ]:
# load ground truths
base_ds = CocoDetection(img_folder='/content/drive/MyDrive/DETR/COCO data/val2017', 
                        ann_file='/content/drive/MyDrive/DETR/COCO data/annotations/instances_val2017.json',
                        feature_extractor=feature_extractor, train=False).coco

In [ ]:
from datasets.coco_eval import CocoEvaluator

iou_types = ['bbox', 'segm']
coco_evaluator = CocoEvaluator(base_ds, iou_types) # initialize evaluator with ground truths

We can use `PanopticEvaluator` to evaluate the panoptic segmentation masks.

! When initializing `PanopticEvaluator`, we should only add the relevant panoptic segmentations (i.e. the annotations for which we are going to evaluate).
Let's only keep the relevant panoptic segmentation annotations:

In [ ]:
import json

# read in all annotations
with open('/content/drive/MyDrive/DETR/COCO data/annotations/coco_panoptic/annotations/panoptic_val2017.json') as f:
  data = json.load(f)

In [ ]:
data.keys()

In [ ]:
# get image ids of images in validation set
image_ids = []
for batch in val_dataloader:
  labels = batch['labels']
  for label in labels:
    image_ids.append(label['image_id'].item())
print(image_ids)

In [ ]:
# only keep those annotations
relevant_annotations = []
for ann in data['annotations']:
  if ann['image_id'] in image_ids:
    relevant_annotations.append(ann)
print(len(relevant_annotations))

In [ ]:
data['annotations'] = relevant_annotations

In [ ]:
# write to json file
import json
with open('panoptic_val.json', 'w') as f:
    json.dump(data, f)

In [ ]:
# check whether this is ok
with open('panoptic_val.json', 'r') as f:
        gt_json = json.load(f)
len(gt_json['annotations'])

In [ ]:
from datasets.panoptic_eval import PanopticEvaluator

# inititialiaze panoptic evaluator with the ground truth annotations
panoptic_evaluator = PanopticEvaluator(
            '/content/detr/panoptic_val.json',
            val_dataloader.dataset.dataset.ann_folder,
            output_dir=".",
        )

Finally, let's evaluate!

In [ ]:
from tqdm.notebook import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.eval()

print("Running evaluation...")

for idx, batch in enumerate(tqdm(val_dataloader)):
    # get the inputs
    pixel_values = batch["pixel_values"].to(device)
    pixel_mask = batch["pixel_mask"].to(device)
    labels = [{k: v.to(device) for k, v in t.items()} for t in batch["labels"]] # these are in DETR format, resized + normalized

    # forward pass
    with torch.no_grad():
      outputs = model.model(pixel_values=pixel_values, pixel_mask=pixel_mask)

    # object detection evaluation
    orig_target_sizes = torch.stack([target["orig_size"] for target in labels], dim=0)
    results = feature_extractor.post_process(outputs, orig_target_sizes) # convert outputs of model to COCO api
    target_sizes = torch.stack([t["size"] for t in labels], dim=0)
    results = feature_extractor.post_process_segmentation(results, outputs, orig_target_sizes, target_sizes)
    res = {target['image_id'].item(): output for target, output in zip(labels, results)}
    coco_evaluator.update(res)

    # panoptic segmentation evaluation
    target_sizes = torch.stack([t["size"] for t in labels], dim=0)
    results_panoptic = feature_extractor.post_process_panoptic(outputs, target_sizes, orig_target_sizes) # convert outputs of model to COCO api
    for i, target in enumerate(labels):
        image_id = target["image_id"].item()
        file_name = f"{image_id:012d}.png"
        results_panoptic[i]["image_id"] = image_id
        results_panoptic[i]["file_name"] = file_name
    panoptic_evaluator.update(results_panoptic)

coco_evaluator.synchronize_between_processes()
coco_evaluator.accumulate()
coco_evaluator.summarize()

In [ ]:
panoptic_evaluator.synchronize_between_processes()
panoptic_evaluator.summarize()

## Inference (+ visualization)

Let's visualize the predictions of DETR on one of the first images of the validation set.

In [ ]:
#We can use the image_id in target to know which image it is
pixel_values, target = val_dataset[1]

In [ ]:
pixel_values = pixel_values.unsqueeze(0).to(device)
print(pixel_values.shape)

In [ ]:
with torch.no_grad():
  # forward pass to get class logits and bounding boxes
  outputs = model(pixel_values=pixel_values, pixel_mask=None)

In [ ]:
import torch

# use the post_process_panoptic method of DetrFeatureExtractor, which expects as input the target size of the predictions (which we set here to the image size)
processed_sizes = torch.as_tensor(pixel_values.shape[-2:]).unsqueeze(0)
result = feature_extractor.post_process_panoptic(outputs, processed_sizes)[0]

In [ ]:
import itertools
import io
import seaborn as sns
import numpy 
from transformers.models.detr.feature_extraction_detr import rgb_to_id
import matplotlib.pyplot as plt

palette = itertools.cycle(sns.color_palette())

# The segmentation is stored in a special-format png
panoptic_seg = Image.open(io.BytesIO(result['png_string']))
panoptic_seg = numpy.array(panoptic_seg, dtype=numpy.uint8).copy()
# We retrieve the ids corresponding to each mask
panoptic_seg_id = rgb_to_id(panoptic_seg)

# Finally we color each mask individually
panoptic_seg[:, :, :] = 0
for id in range(panoptic_seg_id.max() + 1):
  panoptic_seg[panoptic_seg_id == id] = numpy.asarray(next(palette)) * 255
plt.figure(figsize=(15,15))
plt.imshow(panoptic_seg)
plt.axis('off')
plt.show()

Compare to the original image and ground truth segmentation:

In [ ]:
image_id = target['image_id'].item()
image = base_ds.loadImgs(image_id)[0]
img = Image.open(os.path.join('/content/drive/MyDrive/DETR/COCO data/val2017', image['file_name']))
img

In [ ]:
# replace jpg by png
file_name = image['file_name'].replace('.jpg', '.png')
image = Image.open(os.path.join('/content/drive/MyDrive/DETR/COCO data/annotations/coco_panoptic/panoptic_val2017', file_name))
image